In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch Dataset 

In [2]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
data_path = "../../dataset/EthenT/"
postfix = "EthenT"
dim = 72
basis = "6-311++g**"
n_electrons = 16
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))


ind = int(0.8 * len(index))
ind_val = int(0.8 * ind)


molecules = (
    molecules[:ind_val], 
    molecules[ind_val:ind], 
    molecules[ind:]
)

s_triu_norm, mu, std = AbstractDataset.normalize(S)


s_train, p_train, s_test, p_test = split(S, P, ind)
s_train, p_train, s_val, p_val = split(s_train, p_train, ind_val)

# Utilities 

## Calculate Descriptors and extract center blocks 

In [3]:
from SCFInitialGuess.utilities.constants import number_of_basis_functions as N_BASIS
from SCFInitialGuess.descriptors.coordinate_descriptors import \
    AtomicNumberWeighted, Gaussians
from SCFInitialGuess.descriptors.coordinate_descriptors import \
    Gaussians, RADIAL_GAUSSIAN_MODELS, AZIMUTHAL_GAUSSIAN_MODELS, POLAR_GAUSSIAN_MODELS
from SCFInitialGuess.utilities.dataset import extract_triu
from SCFInitialGuess.utilities.dataset import StaticDataset


def make_mask(mol, species):

    masks = []
    current_dim = 0
    for atom in mol.species:
        # calculate block range
        index_start = current_dim
        current_dim += N_BASIS[mol.basis][atom] 
        index_end = current_dim

        if atom == species:

            # calculate logical vector
            L = np.arange(dim)
            L = np.logical_and(index_start <= L, L < index_end)

            masks.append(np.logical_and.outer(L, L))
            
    
    return masks




def extract_dataset(molecules, p_batch, species):    
    
    # make mask to extract central blocks
    masks = make_mask(molecules[0], species)
    
    descriptor = AtomicNumberWeighted(
        Gaussians(*RADIAL_GAUSSIAN_MODELS["Equidistant-Broadening_1"]),
        Gaussians(*AZIMUTHAL_GAUSSIAN_MODELS["Equisitant_1"]),
        Gaussians(*POLAR_GAUSSIAN_MODELS["Equisitant_1"])
    )
    
    descriptor_values, blocks = [], []
    for p, mol in zip(p_batch, molecules):
        for mask in masks:
            blocks.append(extract_triu(
                p.copy()[mask], 
                N_BASIS[mol.basis][species]
            ))
        
        for i, atom in enumerate(mol.species):
            if atom == species:
                descriptor_values.append(
                    descriptor.calculate_atom_descriptor(
                        i, 
                        mol,
                        descriptor.number_of_descriptors
                    )
                )
            
    return descriptor_values, blocks


def make_dataset(species):
    
    inputs_test, outputs_test = extract_dataset(
        molecules[2], 
        p_test.reshape(-1, dim, dim),
        species
    )
    
    inputs_validation, outputs_validation = extract_dataset(
        molecules[1], 
        p_val.reshape(-1, dim, dim),
        species
    )

    inputs_train, outputs_train = extract_dataset(
        molecules[0], 
        p_train.reshape(-1, dim, dim),
        species
    )
    
    
    _, mu, std = StaticDataset.normalize(inputs_train + inputs_validation + inputs_test)
    
    dataset = StaticDataset(
        train=(
            StaticDataset.normalize(inputs_train, mu, std)[0], 
            np.asarray(outputs_train)
        ),
        validation=(
            StaticDataset.normalize(inputs_validation, mu, std)[0], 
            np.asarray(outputs_validation)
        ),
        test=(
            StaticDataset.normalize(inputs_test, mu, std)[0], 
            np.asarray(outputs_test)
        ),
        mu=mu,
        std=std
    )
    
    return dataset

## Networks 

In [27]:
#keras.backend.clear_session()

#activation = "elu"
#learning_rate = 1e-5

intializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)

def make_model(
        structure, 
        input_dim, 
        output_dim,
        activation="elu", 
        learning_rate=1e-5
    ):

    model = keras.Sequential()

    # input layer
    model.add(keras.layers.Dense(
        structure[0], 
        activation=activation, 
        input_dim=input_dim, 
        kernel_initializer=intializer
    ))

    for layer in structure[1:]:
        model.add(keras.layers.Dense(
            layer, 
            activation=activation, 
            kernel_initializer=intializer, 
            #bias_initializer='zeros',
            #kernel_regularizer=keras.regularizers.l2(1e-8)
        ))

    #output
    model.add(keras.layers.Dense(output_dim))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate), 
        loss='MSE', 
        metrics=['mae', 'mse']
    )
    
    return model



In [33]:
filepath = "../../models/ParticleNumberIndependent/CenterBlocks/model_C_" + postfix + ".h5"

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mean_squared_error", 
    min_delta=1e-10, 
    patience=200, 
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_mean_squared_error', 
    factor=0.5, 
    patience=50, 
    verbose=1, 
    mode='auto', 
    min_delta=1e-6, 
    cooldown=50, 
    min_lr=1e-10
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, 
    monitor='val_mean_squared_error', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', 
    period=1
)

epochs = 1000


def train_model(model, dataset, learning_rate=5e-5):
    
    error = []
    while True:
        keras.backend.set_value(model.optimizer.lr, learning_rate)
            
        history = model.fit(
            x = dataset.training[0],
            y = dataset.training[1],
            epochs=epochs,
            shuffle=True,
            validation_data=dataset.validation, 
            verbose=1, 
            callbacks=[
                early_stopping, 
                reduce_lr,
                checkpoint
            ]
        )

        error.append(model.evaluate(
            dataset.testing[0], 
            dataset.testing[1], 
            verbose=1
        )[1])
    
    return error
    

# C  

## Compute inputs 

In [29]:
from SCFInitialGuess.utilities.constants import number_of_basis_functions as N_BASIS


species = "C"

dim_C = N_BASIS[basis][species]
dim_C_triu = dim_C * (dim_C + 1) // 2

dataset_C = make_dataset(species)

In [36]:
structure_C = [dim_C_triu - 50, dim_C_triu + 100, dim_C_triu + 50]

In [37]:
keras.backend.clear_session()
model_C = make_model(
    structure=structure_C,
    input_dim=dataset_C.training[0].shape[1],
    output_dim=dim_C_triu
)

model_C.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 273)               5187      
_________________________________________________________________
dense_1 (Dense)              (None, 353)               96722     
_________________________________________________________________
dense_2 (Dense)              (None, 303)               107262    
_________________________________________________________________
dense_3 (Dense)              (None, 253)               76912     
Total params: 286,083
Trainable params: 286,083
Non-trainable params: 0
_________________________________________________________________


In [19]:

keras.backend.clear_session()

try: 
    model_C = keras.models.load_model(filepath)
except OSError:
    print("Model not found at " + filepath)
    model_C = make_model(
        structure=structure_C,
        input_dim=dataset_C.training[0].shape[1],
        output_dim=dim_C_triu
    )
    
model_C.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 453)               8607      
_________________________________________________________________
dense_1 (Dense)              (None, 353)               160262    
_________________________________________________________________
dense_2 (Dense)              (None, 303)               107262    
_________________________________________________________________
dense_3 (Dense)              (None, 253)               76912     
Total params: 353,043
Trainable params: 353,043
Non-trainable params: 0
_________________________________________________________________


In [38]:
train_model(model_C, dataset_C, learning_rate=5e-4)

Train on 1280 samples, validate on 320 samples
Epoch 1/1000
1280/1280 [==============================] - 1s 437us/step - loss: 0.0032 - mean_absolute_error: 0.0215 - mean_squared_error: 0.0032 - val_loss: 7.3593e-04 - val_mean_absolute_error: 0.0140 - val_mean_squared_error: 7.3593e-04

Epoch 00001: val_mean_squared_error did not improve from 0.00035
Epoch 2/1000
1280/1280 [==============================] - 0s 161us/step - loss: 4.7413e-04 - mean_absolute_error: 0.0115 - mean_squared_error: 4.7413e-04 - val_loss: 4.8854e-04 - val_mean_absolute_error: 0.0107 - val_mean_squared_error: 4.8854e-04

Epoch 00002: val_mean_squared_error did not improve from 0.00035
Epoch 3/1000
1280/1280 [==============================] - 0s 157us/step - loss: 4.1837e-04 - mean_absolute_error: 0.0106 - mean_squared_error: 4.1837e-04 - val_loss: 4.8278e-04 - val_mean_absolute_error: 0.0106 - val_mean_squared_error: 4.8278e-04

Epoch 00003: val_mean_squared_error did not improve from 0.00035
Epoch 4/1000
1280/1

1280/1280 [==============================] - 0s 336us/step - loss: 3.6501e-04 - mean_absolute_error: 0.0099 - mean_squared_error: 3.6501e-04 - val_loss: 4.2582e-04 - val_mean_absolute_error: 0.0099 - val_mean_squared_error: 4.2582e-04

Epoch 00027: val_mean_squared_error did not improve from 0.00035
Epoch 28/1000
1280/1280 [==============================] - 0s 271us/step - loss: 3.6422e-04 - mean_absolute_error: 0.0098 - mean_squared_error: 3.6422e-04 - val_loss: 4.2019e-04 - val_mean_absolute_error: 0.0098 - val_mean_squared_error: 4.2019e-04

Epoch 00028: val_mean_squared_error did not improve from 0.00035
Epoch 29/1000
1280/1280 [==============================] - 0s 255us/step - loss: 3.5721e-04 - mean_absolute_error: 0.0097 - mean_squared_error: 3.5721e-04 - val_loss: 4.3972e-04 - val_mean_absolute_error: 0.0099 - val_mean_squared_error: 4.3972e-04

Epoch 00029: val_mean_squared_error did not improve from 0.00035
Epoch 30/1000
1280/1280 [==============================] - 0s 289us/s

1280/1280 [==============================] - 0s 200us/step - loss: 3.4656e-04 - mean_absolute_error: 0.0096 - mean_squared_error: 3.4656e-04 - val_loss: 4.1049e-04 - val_mean_absolute_error: 0.0096 - val_mean_squared_error: 4.1049e-04

Epoch 00053: val_mean_squared_error did not improve from 0.00035
Epoch 54/1000
1280/1280 [==============================] - 0s 192us/step - loss: 3.4518e-04 - mean_absolute_error: 0.0096 - mean_squared_error: 3.4518e-04 - val_loss: 4.0614e-04 - val_mean_absolute_error: 0.0096 - val_mean_squared_error: 4.0614e-04

Epoch 00054: val_mean_squared_error did not improve from 0.00035
Epoch 55/1000
1280/1280 [==============================] - 0s 173us/step - loss: 3.4177e-04 - mean_absolute_error: 0.0096 - mean_squared_error: 3.4177e-04 - val_loss: 4.0168e-04 - val_mean_absolute_error: 0.0095 - val_mean_squared_error: 4.0168e-04

Epoch 00055: val_mean_squared_error did not improve from 0.00035
Epoch 56/1000
1280/1280 [==============================] - 0s 210us/s

1280/1280 [==============================] - 0s 152us/step - loss: 3.2699e-04 - mean_absolute_error: 0.0093 - mean_squared_error: 3.2699e-04 - val_loss: 4.0065e-04 - val_mean_absolute_error: 0.0096 - val_mean_squared_error: 4.0065e-04

Epoch 00079: val_mean_squared_error did not improve from 0.00035
Epoch 80/1000
1280/1280 [==============================] - 0s 145us/step - loss: 3.3132e-04 - mean_absolute_error: 0.0094 - mean_squared_error: 3.3132e-04 - val_loss: 3.9998e-04 - val_mean_absolute_error: 0.0097 - val_mean_squared_error: 3.9998e-04

Epoch 00080: val_mean_squared_error did not improve from 0.00035
Epoch 81/1000
1280/1280 [==============================] - 0s 150us/step - loss: 3.2541e-04 - mean_absolute_error: 0.0094 - mean_squared_error: 3.2541e-04 - val_loss: 3.9502e-04 - val_mean_absolute_error: 0.0094 - val_mean_squared_error: 3.9502e-04

Epoch 00081: val_mean_squared_error did not improve from 0.00035
Epoch 82/1000
1280/1280 [==============================] - 0s 153us/s

1280/1280 [==============================] - 0s 316us/step - loss: 3.0788e-04 - mean_absolute_error: 0.0091 - mean_squared_error: 3.0788e-04 - val_loss: 3.8238e-04 - val_mean_absolute_error: 0.0093 - val_mean_squared_error: 3.8238e-04

Epoch 00105: val_mean_squared_error did not improve from 0.00035
Epoch 106/1000
1280/1280 [==============================] - 1s 422us/step - loss: 3.0836e-04 - mean_absolute_error: 0.0091 - mean_squared_error: 3.0836e-04 - val_loss: 3.7692e-04 - val_mean_absolute_error: 0.0092 - val_mean_squared_error: 3.7692e-04

Epoch 00106: val_mean_squared_error did not improve from 0.00035
Epoch 107/1000
1280/1280 [==============================] - 0s 337us/step - loss: 3.0710e-04 - mean_absolute_error: 0.0091 - mean_squared_error: 3.0710e-04 - val_loss: 3.8198e-04 - val_mean_absolute_error: 0.0093 - val_mean_squared_error: 3.8198e-04

Epoch 00107: val_mean_squared_error did not improve from 0.00035
Epoch 108/1000
1280/1280 [==============================] - 0s 313u

KeyboardInterrupt: 